In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.1 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
  0% 0.00/91.2M [00:00<?, ?B/s]
100% 91.2M/91.2M [00:00<00:00, 1.44GB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import gc
import os
import sys
from tqdm.auto import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
CONFIG = {
    # Đường dẫn folder chứa file Ankh
    'ANKH_DIR': '/content/drive/MyDrive/CAFA6_Embeddings/Ankh3_Large_Aligned',
    # Đường dẫn folder chứa file ESM (Đã align theo Ankh)
    # 'ESM_DIR':  '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh',

    'SAVE_DIR': '/content/drive/MyDrive/CAFA6_Results/Ankh3_Large_Run',
    'TRAIN_TERMS': '/content/cafa6_data/Train/train_terms.tsv',
    'IA_FILE': '/content/cafa6_data/IA.tsv',
    'TEST_FASTA': '/content/cafa6_data/Test/testsuperset.fasta',

    # Model Params
    'TOP_K_LABELS': 10000, # Giữ mức này để model học tốt
    # 'MIN_FREQ': 3,

    # Input sẽ là 768 (Ankh) + 1280 (ESM) = 2048
    'HIDDEN_DIMS': [2048, 1024, 512],

    'DROPOUT_RATE': 0.3, # Tăng dropout vì input lớn
    'EPOCHS': 40,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 5e-4, # Giảm LR một chút cho ổn định
    'LABEL_SMOOTHING': 0.1,

    # Inference Params (Chuẩn 0.27 điểm)
    'MIN_CONFIDENCE': 0.001,
    'MAX_PREDS_PER_PROTEIN': 20,

    'SEED': 42,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

os.makedirs(CONFIG['SAVE_DIR'], exist_ok=True)
print(f"🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: {CONFIG['DEVICE']}")

torch.manual_seed(CONFIG['SEED'])
np.random.seed(CONFIG['SEED'])

# ============================================================================
# 1. MEMORY-SAFE DATASET (CHÌA KHÓA ĐỂ KHÔNG TRÀN RAM)
# ============================================================================
class DualMemmapDataset(Dataset):
    def __init__(self, ankh_path, y_tensor=None, indices=None):
        # Chỉ map file, không load vào RAM
        self.ankh_mmap = np.load(ankh_path, mmap_mode='r')
        # self.esm_mmap = np.load(esm_path, mmap_mode='r')

        # Nếu dùng indices (cho train/val split)
        self.indices = indices if indices is not None else np.arange(len(self.ankh_mmap))
        self.y = y_tensor

        # Check độ dài
        # assert len(self.ankh_mmap) == len(self.esm_mmap), "Lệch số lượng dòng giữa Ankh và ESM!"

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]

        # Đọc từ đĩa và convert sang Tensor ngay lập tức
        # copy() là bắt buộc để chuyển từ memmap sang RAM array
        a = torch.from_numpy(self.ankh_mmap[real_idx].copy()).float()
        # e = torch.from_numpy(self.esm_mmap[real_idx].copy()).float()

        # Nối vector ngay tại đây (On-the-fly)
        # Không tốn RAM lưu trữ ma trận gộp
        x = a

        if self.y is not None:
            return x, self.y[real_idx]
        return (x,)

# ============================================================================
# 2. LOAD DATA & PROCESS LABELS
# ============================================================================
print("\n[1/5] Checking Files...")
# Định nghĩa đường dẫn file
ankh_train_path = f"{CONFIG['ANKH_DIR']}/ankh3_large_train_FINAL_embeddings.npy"
# esm_train_path = f"{CONFIG['ESM_DIR']}/esm2_train_FINAL_embeddings.npy"
ids_path = f"{CONFIG['ANKH_DIR']}/ankh3_large_train_FINAL_ids.npy"

# Load IDs để map nhãn
train_ids = np.load(ids_path)
print(f"   ✓ Train IDs: {len(train_ids)}")

print("\n[2/5] Processing Labels (IA Strategy)...")
# 1. Load Terms & IA
df_terms = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', header=0, names=['EntryID', 'term', 'aspect'])
df_ia = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'])
ia_dict = dict(zip(df_ia['term'], df_ia['ia']))

# 2. Tính Score
term_counts = df_terms['term'].value_counts().reset_index()
term_counts.columns = ['term', 'freq']

# print(f"   Original terms count: {len(term_counts)}")
# term_counts = term_counts[term_counts['freq'] >= CONFIG['MIN_FREQ']]
# print(f"   -> After filtering (Freq >= {CONFIG['MIN_FREQ']}): {len(term_counts)} terms")

term_counts['ia'] = term_counts['term'].map(ia_dict).fillna(0.0)
term_counts['score'] = term_counts['freq'] * term_counts['ia']

# 3. Chọn Top K
top_terms_df = term_counts.sort_values(by='score', ascending=False).head(CONFIG['TOP_K_LABELS'])
top_terms = top_terms_df['term'].tolist()
print(f"   ✓ Selected {len(top_terms)} terms.")

# 4. Map ID -> Terms
df_filtered = df_terms[df_terms['term'].isin(top_terms)]
id_to_terms = df_filtered.groupby('EntryID')['term'].apply(list).to_dict()

# Dọn dẹp
del df_terms, df_filtered, df_ia, term_counts, top_terms_df
gc.collect()

# ============================================================================
# 3. PREPARE LABELS (RAM OPTIMIZED)
# ============================================================================
print("\n[3/5] Preparing Labels (Sparse Mode)...")

# 1. Sparse MLB
mlb = MultiLabelBinarizer(classes=top_terms, sparse_output=True)
mlb.fit([top_terms])

# 2. Transform -> Sparse Matrix
y_labels_list = [id_to_terms.get(pid, []) for pid in train_ids]
y_train_sparse = mlb.transform(y_labels_list)
del y_labels_list, train_ids # Xóa ID list không dùng nữa
gc.collect()

# 3. Weights (Optional)
# Nếu dùng IA weight thì giữ, nếu muốn model tự học thì comment dòng này và bỏ pos_weight trong Loss
weights_list = [ia_dict.get(t, 0.0) for t in mlb.classes_]
pos_weight_tensor = torch.tensor(weights_list, dtype=torch.float32).to(CONFIG['DEVICE'])

# 4. Convert to Dense Float32 & Label Smoothing
print("   ⏳ Converting Labels to Tensor...")
# Convert từng phần nhỏ hoặc convert hết nếu RAM > 12GB (với 80k row x 10k col float32 ~ 3.2GB -> Ổn)
y_train_binary = y_train_sparse.astype(np.float32).toarray()
y_train_tensor = torch.from_numpy(y_train_binary)

if CONFIG['LABEL_SMOOTHING'] > 0:
    y_train_tensor.mul_(1 - CONFIG['LABEL_SMOOTHING']).add_(CONFIG['LABEL_SMOOTHING'] / len(top_terms))

del y_train_sparse, y_train_binary
gc.collect()

# ============================================================================
# 4. CREATE DATA LOADERS
# ============================================================================
print("\n[4/5] Creating DataLoaders...")

# Split Indices (Chỉ chia index, không chia dữ liệu)
indices = np.arange(len(y_train_tensor))
train_idx, val_idx = train_test_split(indices, test_size=0.15, random_state=CONFIG['SEED'])

# Tạo Dataset đặc biệt (Đọc đĩa)
train_dataset = DualMemmapDataset(ankh_train_path, y_train_tensor, indices=train_idx)
val_dataset = DualMemmapDataset(ankh_train_path, y_train_tensor, indices=val_idx)

# Lấy input dim thực tế để init model
sample_x, _ = train_dataset[0]
INPUT_DIM = sample_x.shape[0]
print(f"   ✓ Combined Input Dim: {INPUT_DIM} (Ankh + ESM)")

train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

# ============================================================================
# 5. MODEL & TRAINING
# ============================================================================
print("\n[5/5] Building Model...")

class DualMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims, dropout):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = dim
        self.encoder = nn.Sequential(*layers)
        self.output = nn.Linear(prev_dim, output_dim)

    def forward(self, x):
        return self.output(self.encoder(x))

model = DualMLP(INPUT_DIM, CONFIG['TOP_K_LABELS'], CONFIG['HIDDEN_DIMS'], CONFIG['DROPOUT_RATE']).to(CONFIG['DEVICE'])

# Loss & Optimizer
# Có thể thử bỏ pos_weight nếu thấy loss dao động quá mạnh
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print(f"\n🚀 START TRAINING ({CONFIG['EPOCHS']} Epochs)...")
best_val_loss = float('inf')
best_model_path = f"{CONFIG['SAVE_DIR']}/ankh3_large_best.pth"

for epoch in range(CONFIG['EPOCHS']):
    model.train()
    train_loss = 0
    for X_b, y_b in train_loader:
        X_b, y_b = X_b.to(CONFIG['DEVICE']), y_b.to(CONFIG['DEVICE'])
        optimizer.zero_grad()
        logits = model(X_b)
        loss = criterion(logits, y_b)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = X_b.to(CONFIG['DEVICE']), y_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            loss = criterion(logits, y_b)
            val_loss += loss.item()

    avg_val = val_loss / len(val_loader)
    scheduler.step(avg_val)

    print(f"Epoch {epoch+1:02d} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | LR: {optimizer.param_groups[0]['lr']:.1e}")

    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print("  ⭐ New Best Model!")

# ============================================================================
# 6. INFERENCE (STREAMING)
# ============================================================================
print("\n🔮 PREDICTING...")

# Dọn dẹp Training
del train_loader, val_loader, train_dataset, val_dataset, y_train_tensor, optimizer
gc.collect()
torch.cuda.empty_cache()

model.load_state_dict(torch.load(best_model_path))
model.eval()

# Load Test IDs & Paths
ankh_test_path = f"{CONFIG['ANKH_DIR']}/ankh3_large_test_FINAL_embeddings.npy"
# esm_test_path = f"{CONFIG['ESM_DIR']}/esm2_test_FINAL_embeddings.npy"
test_ids = np.load(f"{CONFIG['ANKH_DIR']}/ankh3_large_test_FINAL_ids.npy")

# Test Dataset (Dual Memmap)
test_dataset = DualMemmapDataset(ankh_test_path)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE']*2, shuffle=False, num_workers=2)

submission_path = f"{CONFIG['SAVE_DIR']}/submission.tsv"
n_predictions = 0

with open(submission_path, 'w') as f:
    current_idx = 0
    with torch.no_grad():
        for (X_b,) in tqdm(test_loader, desc="Inference"):
            X_b = X_b.to(CONFIG['DEVICE'])
            logits = model(X_b)
            probs_batch = torch.sigmoid(logits).cpu().numpy()

            ids_batch = test_ids[current_idx : current_idx + len(probs_batch)]
            current_idx += len(probs_batch)

            for i, pid in enumerate(ids_batch):
                probs = probs_batch[i]

                # TOP-K CỨNG (Chiến thuật 0.27 điểm)
                top_k = CONFIG['MAX_PREDS_PER_PROTEIN']
                ind = np.argpartition(probs, -top_k)[-top_k:]
                ind = ind[np.argsort(probs[ind])][::-1]

                for idx in ind:
                    score = probs[idx]
                    if score > CONFIG['MIN_CONFIDENCE']:
                        f.write(f"{pid}\t{top_terms[idx]}\t{score:.3f}\n")
                        n_predictions += 1

            del probs_batch, X_b, logits

print(f"\n✅ DONE! File: {submission_path}")
print(f"\n✅ DONE! Predictions: {n_predictions:,}")

🚀 CAFA 6 - DUAL MODEL (ANKH + ESM) | Device: cuda

[1/5] Checking Files...
   ✓ Train IDs: 82404

[2/5] Processing Labels (IA Strategy)...
   ✓ Selected 10000 terms.

[3/5] Preparing Labels (Sparse Mode)...
   ⏳ Converting Labels to Tensor...

[4/5] Creating DataLoaders...
   ✓ Combined Input Dim: 1536 (Ankh + ESM)

[5/5] Building Model...

🚀 START TRAINING (40 Epochs)...
Epoch 01 | Train: 0.0239 | Val: 0.0058 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 02 | Train: 0.0058 | Val: 0.0056 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 03 | Train: 0.0057 | Val: 0.0056 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 04 | Train: 0.0056 | Val: 0.0055 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 05 | Train: 0.0055 | Val: 0.0054 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 06 | Train: 0.0054 | Val: 0.0056 | LR: 5.0e-04
Epoch 07 | Train: 0.0052 | Val: 0.0051 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 08 | Train: 0.0051 | Val: 0.0050 | LR: 5.0e-04
  ⭐ New Best Model!
Epoch 09 | Train: 0.0050 | Val: 0.0050 | LR: 5.0e-04
  ⭐ New 

Inference:   0%|          | 0/877 [00:00<?, ?it/s]


✅ DONE! File: /content/drive/MyDrive/CAFA6_Results/Ankh3_Large_Run/submission.tsv

✅ DONE! Predictions: 4,486,180


In [4]:
# Chạy lệnh này trong một cell mới
!kaggle competitions submit \
    -c cafa-6-protein-function-prediction \
    -f /content/drive/MyDrive/CAFA6_Results/Ankh3_Large_Run/submission.tsv \
    -m "ankh3 large"

100% 103M/103M [00:02<00:00, 52.2MB/s] 
Successfully submitted to CAFA 6 Protein Function Prediction